<a href="https://colab.research.google.com/github/anandkc812/Classification_proj/blob/master/My_Resnet_model_1_cifar100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import the necessary packages
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import add
from keras.regularizers import l2
from keras import backend as K
from keras import backend

import keras
from keras import applications




Using TensorFlow backend.


In [0]:
from keras.datasets import cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')
print('Loading Cifar100 train+test done')


169009152/169001437 [==============================] - 6s 0us/step
Loading Cifar100 train+test done


In [0]:
#from keras.models  import Sequential
#from keras.layers import Dense, Dropout, Activation, Flatten
#from tensorflow.python.keras.applications import ResNet50

from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense


num_classes=100
#------------------BUILD MODEL- 2 with lowernumber of layers---------------------------------

#y_train_cat = keras.utils.to_categorical(y_train, num_classes=num_classes, dtype='float32')
y_train_cat = keras.utils.to_categorical(y_train, num_classes=num_classes, dtype='float32')
y_test_cat = keras.utils.to_categorical(y_test, num_classes=num_classes, dtype='float32')

NUM_CLASSES = 100
#resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
resnet_weights_path = 'imagenet'


#model.layers[0].trainable = False
model = Sequential()

resnet_layer =  ResNet50(include_top = False, pooling = 'avg', weights = resnet_weights_path)


resnet_output = model.add(resnet_layer)

# track the inputs to the first layer
inputs = model.layers[0].inputs

class_layer  = (Dense(NUM_CLASSES, activation = 'sigmoid', name='class_layer'))

outputs = model.add(class_layer)


# Create original model and save it

#model.load_weights(resnet_weights_path)


model.summary()

print('\033[1;37;45m RESNET model- backend  ')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
class_layer (Dense)          (None, 100)               204900    
Total params: 23,792,612
Trainable params: 23,739,492
Non-trainable params: 53,120
_________________________________________________________________
 RESNET model- backend  


In [0]:

from keras.models import load_model
from google.colab import drive
import pandas as pd
import os

#=========================NO AUGMENTATION=============================
#=========================NO AUGMENTATION=============================
#=========================NO AUGMENTATION=============================

drive.mount('/content/gdrive')

weights_file = '/content/gdrive/My Drive/Colab Notebooks/MyCifar100/myresnet_cf100_model_1.h5'
history_file = '/content/gdrive/My Drive/Colab Notebooks/MyCifar100/myresnet_cf100_model_1.csv'

exists = os.path.isfile(weights_file)
if exists:
    # Append mode 
    start_range =2 
    model.load_weights(weights_file)
    print('File exists, load weights....',weights_file)
else:
    #Start New afresh
    print('File Does not exists, start new ')
    start_range =1
    
    

model.compile(loss='categorical_crossentropy', optimizer='sgd',metrics=['accuracy'])



for iter in range(start_range,50):
        print("---------------------Epoch :: ", iter) 
        crn50_ = model.fit(x=x_train, y=y_train_cat, batch_size=32, epochs=1, verbose=1, validation_data=(x_test, y_test_cat), shuffle=True)


        model.save(weights_file)
        print('Completed saving weights iter', iter)

        
        if iter==1:
            pd.DataFrame(crn50_.history).to_csv(history_file, mode='w', header=True)
        else:
            pd.DataFrame(crn50_.history).to_csv(history_file, mode='a', header=False)
        
        print("saved History iter", iter)





In [0]:
from keras.models import load_model
from google.colab import drive
import pandas as pd
import os
from keras.preprocessing.image import ImageDataGenerator

#-------------------WITH DATA AUGMENTATION----------------------------------------------------
drive.mount('/content/gdrive')

weights_file = '/content/gdrive/My Drive/Colab Notebooks/MyCifar100/myresnet_cf100_temp.h5'
history_file = '/content/gdrive/My Drive/Colab Notebooks/MyCifar100/myresnet_cf100_model_3a_aug.csv'
EPOCHS = 2
BS = 32


exists = os.path.isfile(weights_file)
if exists:
    # Append mode 
    start_range =2 
    model.load_weights(weights_file)
    print('File exists, load weights....',weights_file)
else:
    #Start New afresh
    print('File Does not exists, start new ')
    start_range =1
    

 
# construct the training image generator for data augmentation
aug = ImageDataGenerator(rescale=1./255,rotation_range=45, zoom_range=0.4,
	width_shift_range=0.4, height_shift_range=0.4, shear_range=0.25,
	horizontal_flip=True, fill_mode="nearest")
 
testaug = ImageDataGenerator(rescale=1./255)


model.compile(loss='categorical_crossentropy', optimizer='sgd',metrics=['accuracy'])



for iter in range(start_range,50):
        print("---------------------Epoch :: ", iter) 
        # train the network
        #crn50_ = model.fit(x=x_train, y=y_train_cat, batch_size=32, epochs=1, verbose=1, validation_data=(x_test, y_test_cat), shuffle=True)

        crn50_ = model.fit_generator(aug.flow(x_train, y_train_cat, batch_size=BS),
          validation_data=testaug.flow(x_test, y_test_cat, batch_size=BS), steps_per_epoch=len(x_train) // BS,
          epochs=EPOCHS, validation_steps =len(x_test)//BS )
        
        model.save(weights_file)
        print('Completed saving weights iter', iter)

        
        if iter==1:
            pd.DataFrame(crn50_.history).to_csv(history_file, mode='w', header=True)
        else:
            pd.DataFrame(crn50_.history).to_csv(history_file, mode='a', header=False)
        
        print("saved History iter", iter)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
File Does not exists, start new 
---------------------Epoch ::  1
Epoch 1/2
1563/1563 [==============================] - 118s 75ms/step - loss: 4.3864 - acc: 0.0451 - val_loss: 3.9892 - val_acc: 0.0941
Epoch 2/2
1563/1563 [==============================] - 111s 71ms/step - loss: 3.8613 - acc: 0.1073 - val_loss: 3.4915 - val_acc: 0.1785
Completed saving weights iter 1
saved History iter 1
---------------------Epoch ::  2
Epoch 1/2
1563/1563 [==============================] - 109s 70ms/step - loss: 3.4824 - acc: 0.1719 - val_loss: 3.0198 - val_acc: 0.2671
Epoch 2/2
1563/1563 [==============================] - 108s 69ms/step - loss: 3.2011 - acc: 0.2237 - val_loss: 2.8840 - val_acc: 0.2966
Completed saving weights iter 2
saved History iter 2
---------------------Epoch ::  3
Epoch 1/2
1563/1563 [==============================] - 108s 69ms/step - loss: 2.9961 - 

In [0]:

DO not run this code ------for reference only
{model.compile(loss='categorical_crossentropy', optimizer='sgd',metrics=['accuracy'])

#----------------Augmentation 1 --------------------
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")

#----------------Augmentation 2 --------------------
aug = ImageDataGenerator(rotation_range=45, zoom_range=0.4,
	width_shift_range=0.4, height_shift_range=0.4, shear_range=0.25,
	horizontal_flip=True, fill_mode="nearest")
}

#----------------Augmentation 3 --------------------
rescaling
aug = ImageDataGenerator(rescale=1./255,rotation_range=45, zoom_range=0.4,
	width_shift_range=0.4, height_shift_range=0.4, shear_range=0.25,
	horizontal_flip=True, fill_mode="nearest")
 
testaug = ImageDataGenerator(rescale=1./255)

#----------------Augmentation 4 --------------------

